### Imp Features 0.03 %

In [71]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import date
from datetime import datetime
import dateutil
from pandas import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [73]:
csv_files = os.chdir('C:\\Users\\HP\\Desktop\\PHD Hackathon\\Data Science\\PHD Models\\Weather_Features')
weather_features = pd.read_csv('weatherData_full.csv', encoding='latin 1')
holidays_features = pd.read_csv('holidays_features.csv')
economic_features = pd.read_csv('economic_features.csv')
womenClothing_sales = pd.read_csv('womenClothing_sales.csv')

In [74]:
print('womenClothing :', womenClothing_sales.shape)
print('holidays_features :', holidays_features.shape)
print('economic_features :', economic_features.shape)
print('weather_features :', weather_features.shape)

womenClothing : (84, 3)
holidays_features : (81, 6)
economic_features : (96, 18)
weather_features : (96, 83)


## Seperate Train and Test Data

### Test Data

In [75]:
weather_features_test = weather_features[weather_features['Year'] == 2016]
holidays_features_test = holidays_features[holidays_features['Year'] == 2016]
economic_features_test = economic_features[economic_features['Year'] == 2016]

In [76]:
print('holidays_features :', holidays_features_test.shape)
print('economic_features :', economic_features_test.shape)
print('weather_features :', weather_features_test.shape)

holidays_features : (11, 6)
economic_features : (12, 18)
weather_features : (12, 83)


In [77]:
test_data = weather_features_test.merge(holidays_features_test, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')
test_data = test_data.merge(economic_features_test, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')

In [78]:
test_data.tail()

,Year,Month,Temp high (°C)min,Temp high (°C)max,Temp high (°C)mean,Temp high (°C)var,Temp avg (°C)min,Temp avg (°C)max,Temp avg (°C)mean,Temp avg (°C)var,...,Earnings or wages in dollars per hour,AdvertisingExpenses (in Thousand Dollars),Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
7,2016,5,11.0,33.0,21.258065,45.197849,9.0,28.0,17.193548,31.894624,...,25.68,0.0,70.28,0.73,8.422,7.920,755,12.455,3.275,8.50
8,2016,6,19.0,31.0,27.100000,11.127586,16.0,27.0,22.433333,6.322989,...,25.42,0.0,74.10,0.73,9.347,8.588,792,12.165,3.275,9.95
9,2016,7,22.0,36.0,30.129032,11.449462,21.0,31.0,26.064516,6.662366,...,25.53,0.0,79.50,0.73,9.824,9.104,802,15.215,3.275,10.95
10,2016,8,26.0,36.0,30.354839,6.169892,22.0,32.0,26.354839,5.303226,...,25.52,0.0,79.50,-0.99,9.824,9.340,787,15.314,3.275,10.95
11,2016,9,15.0,33.0,26.166667,19.040230,14.0,28.0,22.133333,12.809195,...,25.74,0.0,77.86,-2.99,9.950,9.464,790,15.580,3.275,10.95


In [79]:
test_data.isnull().sum()

Year                                                                 0
Month                                                                0
Temp high (°C)min                                                    0
Temp high (°C)max                                                    0
Temp high (°C)mean                                                   0
Temp high (°C)var                                                    0
Temp avg (°C)min                                                     0
Temp avg (°C)max                                                     0
Temp avg (°C)mean                                                    0
Temp avg (°C)var                                                     0
Temp low (°C)min                                                     0
Temp low (°C)max                                                     0
Temp low (°C)mean                                                    0
Temp low (°C)var                                                     0
Dew Po

### Train Data

In [80]:
weather_features_train = weather_features[weather_features['Year'] != 2016]
holidays_features_train = holidays_features[holidays_features['Year'] != 2016]
economic_features_train = economic_features[economic_features['Year'] != 2016]

In [81]:
print('holidays_features :', holidays_features_train.shape)
print('economic_features :', economic_features_train.shape)
print('weather_features :', weather_features_train.shape)

holidays_features : (70, 6)
economic_features : (84, 18)
weather_features : (84, 83)


In [82]:
train_data = womenClothing_sales.merge(holidays_features_train, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')
train_data = train_data.merge(economic_features_train, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')
train_data = train_data.merge(weather_features_train, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')

In [83]:
train_data.head()

,Year,Month,WomenClothing_Sales,Event,Federal Holiday,Event_Yes_No,Federal Holiday_Yes_No,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,...,Wind (km/h) highvar,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Other,Rain,"Rain , Snow",Snow,Thunderstorm
0,2009,1,1755.0,0.0,2.0,No,Yes,14421.752895,14407.053343,233.402,...,96.625806,0,0,2,2,21,3,0,3,0
1,2009,2,1729.0,1.0,1.0,Yes,Yes,14389.200466,14366.176571,234.663,...,167.349206,0,0,0,1,22,4,1,0,0
2,2009,3,2256.0,NaN,NaN,NaN,NaN,14340.701639,14351.786822,235.067,...,92.245161,1,0,0,2,21,7,0,0,0
3,2009,4,2662.0,1.0,0.0,Yes,No,14326.815525,14351.601731,235.582,...,87.426437,0,1,0,0,18,11,0,0,0
4,2009,5,2732.0,1.0,1.0,Yes,Yes,14345.904809,14368.123959,235.975,...,88.874886,0,4,0,0,17,10,0,0,0


In [84]:
train_data.shape

(84, 104)

In [85]:
train_data.isnull().sum()

Year                                                                  0
Month                                                                 0
WomenClothing_Sales                                                   0
Event                                                                14
Federal Holiday                                                      14
Event_Yes_No                                                         14
Federal Holiday_Yes_No                                               14
Monthly Nominal GDP Index (inMillion$)                                0
Monthly Real GDP Index (inMillion$)                                   0
CPI                                                                   0
unemployment rate                                                     0
CommercialBankInterestRateonCreditCardPlans                           0
Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan     0
Earnings or wages  in dollars per hour                          

In [86]:
# train_data['new_col'] = train_data['Month'].apply(lambda x: 1 if x==12 else 0)
# test_data['new_col'] = test_data['Month'].apply(lambda x: 1 if x==12 else 0)


### Drop Columns

In [87]:
X = train_data.copy().drop(['WomenClothing_Sales','Event_Yes_No', 'Federal Holiday_Yes_No'], axis = 1)
Y = train_data['WomenClothing_Sales']
Y1 = train_data['Year']

### Fill Missing Values

In [88]:
X = X.fillna(0)
test_data = test_data.fillna(0)

## Models

In [89]:
trainX_fold, valX_fold, trainY_fold, valY_fold =  train_test_split(X, Y1, test_size=0.2, random_state=42, shuffle=False)
y_train, y_val = Y.loc[trainY_fold.index], Y.loc[valY_fold.index]

In [90]:
print(trainX_fold.shape)
type(y_train)

(67, 101)


pandas.core.series.Series

### MAPE Function

In [91]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [92]:
def run_lgb(train_X, train_y, val_X, val_y):
    param = {'num_leaves': 30,
#              'min_data_in_leaf': 100,
             'objective': 'regression',
             'max_depth': 8,
             'learning_rate': 0.01,
             "boosting": "gbdt",
             "feature_fraction": 0.7,
             "bagging_freq": 1,
             "bagging_fraction": 0.7,
             "bagging_seed": 11,
             "metric": 'mape',
             # "lambda_l1": 0.2634,
             "random_state": 133,
             "verbosity": -1}

    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(param, lgtrain, 2000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100,
                      evals_result=evals_result)
    return model, evals_result


In [93]:
model, evals_result = run_lgb(trainX_fold, y_train, valX_fold, y_val)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's mape: 0.186969
[200]	valid_0's mape: 0.176913
[300]	valid_0's mape: 0.170845
[400]	valid_0's mape: 0.16492
[500]	valid_0's mape: 0.160723
[600]	valid_0's mape: 0.158184
[700]	valid_0's mape: 0.156507
[800]	valid_0's mape: 0.155842
[900]	valid_0's mape: 0.153278
[1000]	valid_0's mape: 0.15227
Early stopping, best iteration is:
[995]	valid_0's mape: 0.151744


### Predict On Test

In [94]:
test_data = test_data.drop(['Event_Yes_No', 'Federal Holiday_Yes_No'], axis=1)

In [95]:
model.predict(test_data)

array([2395.84864769, 2957.35444629, 2957.35444629, 2707.55445066,
       2395.84864769, 2657.30311592, 2605.57088298, 2769.38714259,
       3055.11425564, 3186.85298939, 3229.47333761, 3039.93684907])

In [96]:
test_results = pd.DataFrame({'Sales(In ThousandDollars)': model.predict(test_data)})

In [97]:
test_results

,Sales(In ThousandDollars)
0,2395.848648
1,2957.354446
2,2957.354446
3,2707.554451
4,2395.848648
5,2657.303116
6,2605.570883
7,2769.387143
8,3055.114256
9,3186.852989


In [98]:
submission = pd.read_csv('C:\\Users\\HP\\Desktop\\PHD Hackathon\\Data Science\\NewSampleSubmission.csv')
submission.head()

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,110,2016,1,WomenClothing
1,2,140,2016,2,WomenClothing
2,3,199,2016,3,WomenClothing
3,4,246,2016,4,WomenClothing
4,5,66,2016,5,WomenClothing


In [99]:
submission['Sales(In ThousandDollars)'] = test_results['Sales(In ThousandDollars)']

In [100]:
submission

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,2395.848648,2016,1,WomenClothing
1,2,2957.354446,2016,2,WomenClothing
2,3,2957.354446,2016,3,WomenClothing
3,4,2707.554451,2016,4,WomenClothing
4,5,2395.848648,2016,5,WomenClothing
5,6,2657.303116,2016,6,WomenClothing
6,7,2605.570883,2016,7,WomenClothing
7,8,2769.387143,2016,8,WomenClothing
8,9,3055.114256,2016,9,WomenClothing
9,10,3186.852989,2016,10,WomenClothing


In [101]:
#submission.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_lgbm_submission.csv', index=False)

In [102]:
############################# ---------------------------------------------------------- #################################

## Random Forest

In [103]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=3,
                                   max_features='sqrt', max_leaf_nodes=None,
                                   min_samples_leaf=2, min_samples_split=3, n_estimators=20, n_jobs=1,)

rf_model.fit(trainX_fold , y_train)

y_pred = rf_model.predict(valX_fold)
mean_absolute_percentage_error(y_val, y_pred)

17.748431136906362

### Take Imp_Features

In [104]:
X_cols = trainX_fold.columns
feature_imp_RFC = pd.DataFrame({"imp_features":rf_model.feature_importances_, "columns":X_cols})
feature_imp_RFC.sort_values(by = 'imp_features', ascending=False)

,columns,imp_features
6,CPI,0.111353
10,Earnings or wages in dollars per hour,0.094199
2,Event,0.081157
1,Month,0.058282
7,unemployment rate,0.050889
5,Monthly Real GDP Index (inMillion$),0.048904
8,CommercialBankInterestRateonCreditCardPlans,0.044108
12,Cotton Monthly Price - US cents per Pound(lbs),0.037165
4,Monthly Nominal GDP Index (inMillion$),0.035246
36,Dew Point avg (°C)min,0.031280


In [105]:
top_features = feature_imp_RFC[feature_imp_RFC['imp_features'] >= 0.03]
top_features.head()

,columns,imp_features
1,Month,0.058282
2,Event,0.081157
4,Monthly Nominal GDP Index (inMillion$),0.035246
5,Monthly Real GDP Index (inMillion$),0.048904
6,CPI,0.111353


In [106]:
top_features.shape

(10, 2)

#### Selecting Imp Features From RF

In [107]:
imp_columns_train = pd.DataFrame()
for i in top_features['columns'].values:
    imp_columns_train[i] = trainX_fold[i]
    
imp_columns_train.head()

,Month,Event,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,CommercialBankInterestRateonCreditCardPlans,Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Dew Point avg (°C)min
0,1,0.0,14421.752895,14407.053343,233.402,7.8,12.03,22.05,57.70,-20.0
1,2,1.0,14389.200466,14366.176571,234.663,8.3,12.97,22.22,55.21,-18.0
2,3,0.0,14340.701639,14351.786822,235.067,8.7,12.97,22.22,51.50,-18.0
3,4,1.0,14326.815525,14351.601731,235.582,9.0,12.97,22.13,56.78,-9.0
4,5,1.0,14345.904809,14368.123959,235.975,9.4,13.32,22.04,61.95,0.0


In [108]:
imp_columns_val = pd.DataFrame()

for i in top_features['columns'].values:
    imp_columns_val[i] = valX_fold[i]

imp_columns_val.head()

,Month,Event,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,CommercialBankInterestRateonCreditCardPlans,Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Dew Point avg (°C)min
67,8,0.0,17684.635349,16196.420132,261.075,6.2,11.82,24.32,74.00,9.0
68,9,0.0,17630.354586,16133.049427,261.074,5.9,11.82,24.50,73.38,6.0
69,10,1.0,17709.436967,16196.814474,260.500,5.7,11.82,24.52,70.34,0.0
70,11,0.0,17765.337570,16243.054734,259.382,5.8,11.99,24.78,67.53,-14.0
71,12,2.0,17733.024464,16221.180125,258.080,5.6,11.99,24.59,68.30,-13.0


### train Model with Imp_Features

In [109]:
rf_model.fit(imp_columns_train, y_train)
y_pred1 = rf_model.predict(imp_columns_val)

mean_absolute_percentage_error(y_val, y_pred1)

11.495648657032659

### Predict Test Data

In [110]:
imp_columns_test = pd.DataFrame()

for i in top_features['columns'].values:
    imp_columns_test[i] = test_data[i]

imp_columns_test.head()

,Month,Event,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,CommercialBankInterestRateonCreditCardPlans,Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Dew Point avg (°C)min
0,1,0.0,18286.638523,16546.153123,260.342,4.9,12.22,25.50,68.75,-19.0
1,10,1.0,18740.780023,16725.460098,264.738,4.8,12.51,26.04,78.52,-4.0
2,11,1.0,18960.461568,16910.896330,265.203,4.6,12.41,25.87,78.92,-8.0
3,12,2.0,19015.393408,16918.050624,265.421,4.7,12.41,25.90,79.50,-18.0
4,2,1.0,18241.713812,16503.500358,260.875,4.9,12.31,25.49,66.57,-27.0


In [111]:
rf_model.predict(imp_columns_test)

array([3628.675   , 3842.040625, 3842.040625, 4118.8     , 3566.175   ,
       3825.790625, 3925.940625, 3925.940625, 3842.040625, 3842.040625,
       3842.040625, 3842.040625])

In [112]:
test_predictions = pd.DataFrame({'Sales(In ThousandDollars)': rf_model.predict(imp_columns_test)})
test_predictions.head(2)

,Sales(In ThousandDollars)
0,3628.675000
1,3842.040625


In [113]:
submissionfile = pd.read_csv('C:\\Users\\HP\\Desktop\\PHD Hackathon\\Data Science\\NewSampleSubmission.csv')
submissionfile.head(2)

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,110,2016,1,WomenClothing
1,2,140,2016,2,WomenClothing


In [114]:
submissionfile['Sales(In ThousandDollars)'] = test_predictions['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,3628.675000,2016,1,WomenClothing
1,2,3842.040625,2016,2,WomenClothing
2,3,3842.040625,2016,3,WomenClothing
3,4,4118.800000,2016,4,WomenClothing
4,5,3566.175000,2016,5,WomenClothing
5,6,3825.790625,2016,6,WomenClothing
6,7,3925.940625,2016,7,WomenClothing
7,8,3925.940625,2016,8,WomenClothing
8,9,3842.040625,2016,9,WomenClothing
9,10,3842.040625,2016,10,WomenClothing


In [115]:
#submissionfile.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_rf_submission.csv', index=False)

### RF Tuning

In [117]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [3, 5, 8, 10, 12],
    'max_features': ['sqrt', 'log2', 'auto'],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [3, 5, 8, 10, 12],
    'n_estimators': [20, 30, 40, 50]
}

rf = RandomForestRegressor()
rf_grid = GridSearchCV(rf, param_grid = param_grid, cv = 5, n_jobs = -1)
rf_grid.fit(imp_columns_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [3, 5, 8, 10, 12], 'max_features': ['sqrt', 'log2', 'auto'], 'min_samples_leaf': [2, 3, 4, 5], 'min_samples_split': [3, 5, 8, 10, 12], 'n_estimators': [20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [118]:
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 8,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 20}

### Build RF Model With Best Parameters

In [119]:
rf_model_best = RandomForestRegressor(bootstrap=True, max_depth=10, max_features='auto', min_samples_leaf=2, 
                                     min_samples_split=5, n_estimators=50)

rf_model_best.fit(imp_columns_train, y_train)
preds = rf_model_best.predict(imp_columns_val)
mean_absolute_percentage_error(y_val, preds)

10.563989420007662

In [120]:
test_preds_rf = pd.DataFrame({'Sales(In ThousandDollars)': rf_model_best.predict(imp_columns_test)})
test_preds_rf

,Sales(In ThousandDollars)
0,3217.482556
1,3889.884247
2,3893.606330
3,4526.391095
4,3413.954794
5,3953.698199
6,3956.351532
7,3968.246699
8,3889.134354
9,3762.213187


In [121]:
submissionfile['Sales(In ThousandDollars)'] = test_preds_rf['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,3217.482556,2016,1,WomenClothing
1,2,3889.884247,2016,2,WomenClothing
2,3,3893.606330,2016,3,WomenClothing
3,4,4526.391095,2016,4,WomenClothing
4,5,3413.954794,2016,5,WomenClothing
5,6,3953.698199,2016,6,WomenClothing
6,7,3956.351532,2016,7,WomenClothing
7,8,3968.246699,2016,8,WomenClothing
8,9,3889.134354,2016,9,WomenClothing
9,10,3762.213187,2016,10,WomenClothing


In [122]:
#submissionfile.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_rf_best_submission.csv', index=False)

In [123]:
##################################################################################################################

### GBM

In [124]:
from sklearn.ensemble import GradientBoostingRegressor

model_gbr = GradientBoostingRegressor()
model_gbr.fit(trainX_fold, y_train)

val_predictions = model_gbr.predict(valX_fold)
mean_absolute_percentage_error(y_val, val_predictions)

10.654107246405015

In [125]:
test_preds = pd.DataFrame({'Sales(In ThousandDollars)': model_gbr.predict(test_data)})
test_preds

,Sales(In ThousandDollars)
0,2347.916693
1,2677.623734
2,2620.298192
3,2805.264416
4,2347.916693
5,2564.596962
6,2539.653033
7,2673.937272
8,2819.600122
9,2858.893428


### GBM With rf_model Imp_features

In [126]:
from sklearn.ensemble import GradientBoostingRegressor

model_gbr = GradientBoostingRegressor()
model_gbr.fit(imp_columns_train, y_train)

val_predictions = model_gbr.predict(imp_columns_val)
mean_absolute_percentage_error(y_val, val_predictions)

9.412698767389585

In [127]:
test_preds1 = pd.DataFrame({'Sales(In ThousandDollars)': model_gbr.predict(imp_columns_test)})

In [128]:
submissionfile['Sales(In ThousandDollars)'] = test_preds1['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,3336.715172,2016,1,WomenClothing
1,2,3989.714962,2016,2,WomenClothing
2,3,4058.782547,2016,3,WomenClothing
3,4,4546.901471,2016,4,WomenClothing
4,5,3534.413837,2016,5,WomenClothing
5,6,4089.684732,2016,6,WomenClothing
6,7,4101.289772,2016,7,WomenClothing
7,8,4203.335271,2016,8,WomenClothing
8,9,3946.348184,2016,9,WomenClothing
9,10,3724.336012,2016,10,WomenClothing


In [129]:
#submissionfile.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_gbm_submission.csv', index=False)

### GBM With GridSearch

In [130]:
parameters = {
    "criterion": ["friedman_mse",'mae'],
    "loss":['ls', 'lad', 'huber', 'quantile'],
    "learning_rate": [0.025, 0.05, 0.075, 0.1, 0.15],
    "max_depth":[3,5,8],
    "n_estimators":[10, 15, 20],
    "min_samples_split":[2,3,4,5,6]
    }

gbc = GradientBoostingRegressor()
gbc_grid = GridSearchCV(gbc, parameters, cv=5, n_jobs=-1)
gbc_grid.fit(imp_columns_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'criterion': ['friedman_mse', 'mae'], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'learning_rate': [0.025, 0.05, 0.075, 0.1, 0.15], 'max_depth': [3, 5, 8], 'n_estimators': [10, 15, 20], 'min_samples_split': [2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [131]:
gbc_grid.best_params_

{'criterion': 'friedman_mse',
 'learning_rate': 0.15,
 'loss': 'ls',
 'max_depth': 8,
 'min_samples_split': 6,
 'n_estimators': 20}

In [132]:
gbc_best = GradientBoostingRegressor(criterion='friedman_mse',
                                        learning_rate = 0.15,
                                        loss = 'ls',
                                        max_depth = 8,
                                        min_samples_split = 4,
                                        n_estimators =20)
gbc_best.fit(imp_columns_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.15, loss='ls', max_depth=8, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=4, min_weight_fraction_leaf=0.0,
             n_estimators=20, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [133]:
y_cap = gbc_best.predict(imp_columns_val)
mean_absolute_percentage_error(y_val, y_cap)

10.407515097957361

In [134]:
gbc_best.predict(imp_columns_test)

array([3111.35839561, 3811.44330354, 3811.44330354, 4448.2538244 ,
       3723.09530219, 4295.7609695 , 4295.7609695 , 4348.27474815,
       3805.71871111, 3652.67867831, 3652.67867831, 3784.31871111])

In [135]:
test_preds2 = pd.DataFrame({'Sales(In ThousandDollars)': gbc_best.predict(imp_columns_test)})
test_preds2

,Sales(In ThousandDollars)
0,3111.358396
1,3811.443304
2,3811.443304
3,4448.253824
4,3723.095302
5,4295.760970
6,4295.760970
7,4348.274748
8,3805.718711
9,3652.678678


In [ ]:
submissionfile['Sales(In ThousandDollars)'] = test_preds2['Sales(In ThousandDollars)']
submissionfile

In [ ]:
#submissionfile.to_csv('C:\\Users\\HP\Desktop\\trial\\regression_gbm_best_submission.csv', index=False)

In [ ]:
######################################################